# Project 12: Spatial fairness in urban environments <br>

In [36]:
import geopandas as gpd
import folium
import numpy as np
import networkx as nx
import pandas as pd
import itertools
import warnings
import sumolib
import os
import openrouteservice as ors
from sklearn.preprocessing import MinMaxScaler

from shapely.geometry import LineString, Point

import warnings
warnings.filterwarnings("ignore")

from plot_utils import edges_to_gps_list, plot_paths, plot_bar_paths_cost
from routing_utils import from_sumo_to_igraph_network
from routing_algorithms import *
import matplotlib.pyplot as plt

attribute = 'traveltime'

### Utility <br>
Functions to improve readability

In [2]:
# modified version of the function of GiulianoCornacchia from the github,
# the only change is that the dataframe has one column for the edge id and one column for the geometry
def gdf_from_sumo_network(sumo_road_network, crs="EPSG:4326", edge_list=None):
    edges_to_plot = sumo_road_network.getEdges()
        
    list_line_strings = []
    list_ids = [] 

    for edge in edges_to_plot: 
        list_ids.append(edge.getID())      
        edge_gps_list = [sumo_road_network.convertXY2LonLat(x[0], x[1]) for x in edge.getShape()]
        list_line_strings.append(LineString(edge_gps_list))

    df_net = gpd.GeoDataFrame(data = list_ids, geometry=list_line_strings, crs="EPSG:4326") 

    df_net.columns = ['id', 'geometry']
    return df_net 

In [3]:
# function to print the correct row for the file xml
# path: tuple/list of edges
# i: identifier for the row
# type: type of trip, in this case if it is toward the city center or towards a random edge
# return: row ready to be written in the file
def path_to_xml(path, i, type):
    edges = ''
    depart = np.random.randint(0, 200)

    for e in path:
        edges+= str(e.getID()) + ' '

    s = '<vehicle id="'+type+str(i)+'" type="veh_passenger" depart="0" departLane="best"> <route edges="'+ edges +'"/> </vehicle> '
    return s

# function to print the correct row for the file xml
# path: list of edges ids
# i: identifier for the row
# type: type of trip, in this case if it is toward the city center or towards a random edge
# return: row ready to be written in the file
def list_to_xml(path, i, type):
    edges = ''
    depart = np.random.randint(0, 200)

    for e in path:
        edges+= str(e) + ' '

    s = '<vehicle id="'+type+str(i)+'" type="veh_passenger" depart="0" departLane="best"> <route edges="'+ edges +'"/> </vehicle> '
    return s

In [4]:
# create a blank map for the city of Leuven
def Leuven_map():
    point = (50.849738563227824, 4.644786576420597)
    m = folium.Map(location=point, zoom_start = 12)
    return m

m = Leuven_map()

## Data

In [5]:
# Open the hubs dataframe
url = 'https://storageaccount11111111.blob.core.windows.net/container1/Leuven/hub_data_leuven/mobility_hubs.gpkg'
hubs = gpd.read_file(url).to_crs('epsg:4326')
folium.GeoJson(hubs).add_to(m)

In [6]:
# Open the zone dataframe
zones_leuven = gpd.read_file('https://storageaccount11111111.blob.core.windows.net/container1/Leuven/socio_demographic_data/leuven_statsec.gpkg')
zones_leuven = zones_leuven.to_crs('epsg:4326')
folium.GeoJson(zones_leuven).add_to(m)

In [40]:
# NORMALIZATION BY AREA
# area for zones in squared meters
utm = zones_leuven.estimate_utm_crs()
zones_leuven['area'] = zones_leuven.to_crs(utm).area

scaler = MinMaxScaler()
zones_leuven['area'] = scaler.fit_transform(np.array(zones_leuven['area']).reshape(-1,1))

In [7]:
# road network from the sumo network
#
# python tools\net\net2geojson.py -n C:\Users\and99\Desktop\Geospatial-project\Sumo\osm.net.xml\osm.net.xml -o road_network.geojson
# road_net = gpd.read_file('road_network_pedestrian.geojson')
#
# build the network, the gdf and the graph directly from the sumo configuration
net = sumolib.net.readNet(os.getcwd()+'\Sumo\osm.net.xml\osm.net.xml')
road_net = gdf_from_sumo_network(net)
G = from_sumo_to_igraph_network(net)

Now some operations are done on the road dataframe. First, for each zone it's useful to have its zone. For the road without zone, I decided to drop them.<br>
Then, the opposite, it's easier to have zones with at least one represented road, so I decided to drop from the zones the one without a road. <br>
Lastly, I decided to extract, for each zone, its neighbors. Since one of the task is to retrieve the time needed for a user to exit from the neighborhood, <br>
it's easier to have a list of the adjecent zones ready at the use.

In [8]:
#Popolate the geodataframe assigning, for each street, its zone
zone_by_street = []

for idx, row in road_net.iterrows():
    street = row['geometry']
    # some streets are outside the zones, this generate an error
    try:
        zone_row = zones_leuven['CODSEC'][np.where(zones_leuven['geometry'].contains(street))[0][0]]
        zone_by_street.append(zone_row)
    except:
        zone_by_street.append('NONE')

road_net['ZONE'] = zone_by_street

# drop all the 'NONE', street, all the street without a zone
road_net.drop(list(np.where(road_net['ZONE'] == 'NONE')[0]), axis = 0, inplace=True)

In [9]:
# in the same way, some of the zone may be useless, since they don't 'point' to any street
selected_zones = set(road_net['ZONE'].tolist())
indexs_to_drop = []

for idx, row in zones_leuven.iterrows():
    if row['CODSEC'] not in selected_zones:
        indexs_to_drop.append(idx)

zones_leuven.drop(indexs_to_drop, inplace=True)
zones_leuven.reset_index(inplace=True)

In [10]:
# retrieve adjacent zones, by zone
adjacent_zones = []

for idx, row in zones_leuven.iterrows():
    zones_row = []
    ids = set(np.where(zones_leuven['geometry'].intersection(zones_leuven['geometry'][idx]))[0]) - set([idx]) # remove the current zone from the list
    for id in ids:
        zones_row.append(zones_leuven['CODSEC'][id])

    adjacent_zones.append(zones_row)

zones_leuven['adjacent'] = adjacent_zones

In [11]:
folium.GeoJson(road_net).add_to(m)
m

In [25]:
traffico = gpd.read_file('traffic_model_leuven_baseline_2019.gpkg')

In [26]:
traffic_df= traffico[['from_node_id', 'to_node_id', 'flows_pae_aggr', 'flows_car_aggr']]
traffic_df.rename(columns={'from_node_id':'origin', 'to_node_id': 'destination'}, inplace=True)
traffic_df.head()

,origin,destination,flows_pae_aggr,flows_car_aggr
0,0,13,386.516271,199.315802
1,13,0,386.516271,199.315802
2,7127,0,3.729844,3.509524
3,0,7127,3.729844,3.509524
4,4425,0,391.115533,203.864369


# Create Traffic demand <br>


### With routing

In [18]:
#init the connection
def get_routing_points(coordinates):
        client = ors.Client(key="5b3ce3597851110001cf624860e7876918cf42f192c5024e4aa25d48")

        # make the query to OSM
        route_osm = client.directions(
                coordinates=coordinates,
                profile='foot-walking',
                preference="recommended",
                format='geojson',
                geometry_simplify=False,
                validate=True)

        # extract the GPS points (lon, lat) of the suggested route
        gps_points_osm = [coord for coord in route_osm['features'][0]['geometry']['coordinates']]
        return gps_points_osm


def convert_in_sumo(gps_points_osm, net):
    edges_osm = []

    for p in gps_points_osm:
        
        # we use sumolib to get the closest edge
        # project p (lon, lat) into SUMO coordinates (x, y)
        x, y = net.convertLonLat2XY(p[0], p[1])
        
        # retrieve the set of Neighboring edges
        candiates_edges = net.getNeighboringEdges(x, y, r=25, includeJunctions=True)

        # IMPORTANT! the result of net.getNeighboringEdges is not sorted by distance
        edges_and_dist = sorted(candiates_edges, key = lambda x: x[1])
        try:
            # take the closest edge ID
            matched_edge = edges_and_dist[0][0].getID()
            # append the edge to the list
            edges_osm.append(matched_edge)
        except:
            continue

        pivot_edges = [(k, sum(1 for _ in vs)) for k, vs in itertools.groupby(edges_osm)]
        pivot_edges = [elem[0] for elem in pivot_edges if elem[1]>1]
        
    return set(edges_osm)

In [ ]:
i = 0
random_destination = ''
center_destination = ''

while i < 100:
    #get a random mobility hub -- starting point
    origin = hubs.sample(1)['geometry'].values[0]
    #coordinate of the origins
    x_o, y_o = origin.x, origin.y
    
    #get its zone
    start_zone = zones_leuven['CODSEC'][np.where(zones_leuven['geometry'].contains(origin))[0][0]] 
    #select one zone of its neighbor zone
    neighbors = zones_leuven[zones_leuven['CODSEC'] == start_zone]['adjacent'].values[0]
    dest_zone = np.random.choice(neighbors)
    
    #destination point 1 -- random street in another zone
    destination = road_net[road_net['ZONE'] == dest_zone].sample(1).centroid.get_coordinates()
    x_d, y_d = float(destination.x), float(destination.y)
    
    coordinates = [[x_o, y_o ], [x_d, y_d]]
    osm_points = get_routing_points(coordinates)
    sumo_edges = convert_in_sumo(osm_points, net)

    if len(sumo_edges) > 1:
        random_destination += list_to_xml(sumo_edges, i, 'r') + '\n'
        i += 1


## Graph Version of the routing <br>
To speed the computation, one version with graph randomization (heavier) and one with shortest path (cheaper)

In [12]:
# Create a traffic demand for the sumo network. 
# Starting from an hub configuration dataframe (points) compute two types of trips:
#   1) towards a random point outside its neighborhood (defined as one random edge in the adjacent zones)
#   2) towards the city center (defined ad-hoc)
# The process is repeated n (default = 50) times and in the end all the trips are written in the xml file in the Sumo folder.
# In this case, I decided to create all vehicles of the same types, the speed is set to
# Currently, the paths are computed as shortest path or, with a higher computational cost, using path randomization. 
# @input hubs: geodataframe with the hubs (points) configuration
# @input fast: boolean value to decide if use shorted past or random graph
# @input n: number of trips to generate

def create_demand(hubs, fast = False, n = 50):

    # city center information
    city_center = (4.70110, 50.87993)
    x_c, y_c = net.convertLonLat2XY(city_center[0], city_center[1])
    candidates_center = net.getNeighboringEdges(x_c, y_c, r=25, includeJunctions=True)
    e_center = sorted(candidates_center, key = lambda x: x[1])[0]

    i = 0
    random_destination = ''
    center_destination = ''

    print('start generation phase...', end='')
    
    while i < n:
        #get a random mobility hub -- starting point
        origin = hubs.sample(1)['geometry'].values[0] 
        
        #get its zone
        start_zone = zones_leuven['CODSEC'][np.where(zones_leuven['geometry'].contains(origin))[0]].values[0]
        #select one zone of its neighbor zone
        neighbors = zones_leuven[zones_leuven['CODSEC'] == start_zone]['adjacent'].values[0]
        dest_zone = np.random.choice(neighbors)
        
        #destination point 1 -- random street in another zone
        destination = road_net[road_net['ZONE'] == dest_zone].sample(1)['id'].values[0]
        #destination point2 -- city center

        #nearest neighbouring edge of the origin
        x_o, y_o = net.convertLonLat2XY(origin.x, origin.y)
        candidates_origin = net.getNeighboringEdges(x_o, y_o, r=25, includeJunctions=True)
        if len(candidates_origin) > 0:
            #origin edge
            e_origin = sorted(candidates_origin, key = lambda x: x[1])[0]
        else:
            continue #shouldn't happen, every hub should have a neighbouring edge
        
        if fast:
            # shortest path to the random destination
            random_path = net.getOptimalPath(e_origin[0], net.getEdge(destination), fastest=False)[0]
            #print('t')
            #shortest path to the city center
            center_path = net.getOptimalPath(e_origin[0], net.getEdge(destination), fastest=False)[0]

            if random_path is not None and center_path is not None :
                random_destination+= path_to_xml(random_path, i, 'r') + " \n"
                center_destination+= path_to_xml(center_path, i, 'c') + " \n"
                i += 1
            else:
                continue

        else:
            # randomized path to the random destination
            gr_paths = graph_randomization(G, e_origin[0].getID(), destination, 3, 1, .01, attribute)
            random_path = np.random.choice(gr_paths)['edges'] # chose one path at random
            #print('t')

            #randomized path to the city center
            gr_paths = graph_randomization(G, e_center[0].getID(), destination, 3, 1, .01, attribute)
            center_path = np.random.choice(gr_paths)['edges']

            if len(random_path) > 0 and len(center_path) > 0 :
                random_destination+= list_to_xml(random_path, i, 'r') + " \n"
                center_destination+= list_to_xml(center_path, i, 'c') + " \n"
                i += 1
            else:
                continue

    
    print('end')
    print('start writing file...', end='')
    # write the file
    f = open("Sumo\my_trips.xml", "w")
    f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?> <routes> <vType id=\"veh_passenger\" vClass=\"passenger\" />")
    f.write(random_destination)
    f.write(center_destination)
    f.write('</routes>')
    f.close()
    print('end')
    

## Duarouter

In [13]:
import subprocess

In [14]:
command_str = "duarouter --route-files Sumo\my_trips.xml --net-file Sumo\osm.net.xml\osm.net.xml --output-file Sumo/traffic_demand_duarouter.rou.xml"

# SUMO

In [15]:
# import the traci library
import libsumo
import sys

In [42]:
# check sumo enviroment
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

#Configuration 
sumo_binary = os.environ['SUMO_HOME']+"/bin/sumo"

#command for the simulation -- use duarouter
sumo_cmd = [sumo_binary, "-c", os.getcwd()+'\Sumo\my_config_duarouter.sumocfg']

In [20]:
# Main loop 
# generate a traffic demand, call duarouter for things, run the simulation. Repeat the process n times and 
# compute the average times.
# @input: hubs, geodataframe with the hub configuration
# @output: random_average_time, average time, between all the simulation and all the different users, starting
#           from a random hub to exit from its neighborhood
# @output: center_average_time, average time, between all the simulation and all the different users, starting
#           from a random hub to reach the city center

def main_loop(hubs):
    random_average_time = []
    center_average_time = []

    #libsumo.vehicle.subsc

    for i in range(10):
        print('-------------')
        print('iteration ', i+1)

        #create random traffic demand
        create_demand(hubs, fast = True)

        #duarouter
        print('start duarouter...', end= '')
        p = subprocess.Popen(command_str, shell=True, stdout=subprocess.PIPE, 
                                        stderr=subprocess.STDOUT)
        p.wait()
        print('end')

        #start new simulation
        print('start simulation...', end= '')
        libsumo.start(sumo_cmd)

        random_time = dict()
        center_time = dict()

        #run simulation until no more vehicles
        while libsumo.simulation_getTime() < 1000 or len(libsumo.vehicle.getIDList()) > 0 :
            libsumo.simulationStep()

            #get, for each vehicle, its last travel time, divided by destinaton
            for v in libsumo.vehicle.getIDList():
                distance = libsumo.vehicle.getDistance(v)
                time = libsumo.simulation_getTime()
                if distance == 0:
                    distance = 1

                timebydist = time/distance
                if v[0] == 'r':
                    #normalize the time based on the distance
                    random_time[v] = timebydist
                else:
                    center_time[v] = timebydist


        random_average_time.append(np.mean(list(random_time.values()))/60)
        center_average_time.append(np.mean(list(center_time.values()))/60)
        print('end')

    #close the simulation
    libsumo.close()
    return np.mean(random_average_time), np.mean(center_average_time)

## Real HUBS

In [21]:
random_mean, center_mean = main_loop(hubs)

-------------
iteration  1
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  2
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  3
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  4
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  5
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  6
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  7
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  8
start generation phase...end
start writing file...end
start duarouter...end
start simulatio

In [22]:
random_mean, center_mean

(0.0021467249412026996, 0.002304316019036832)

## Random Hubs

In [48]:
# random hubs --> random street, centroid
np.random.seed(42)
r_hubs = road_net.sample(len(hubs), replace= True)

In [49]:
# use points instead of edges
coord = [line.centroid for line in r_hubs['geometry']]
r_hubs['geometry'] = coord

In [64]:
random_mean, center_mean = main_loop(r_hubs)

-------------
iteration  1
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  2
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  3
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  4
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  5
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  6
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  7
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  8
start generation phase...end
start writing file...end
start duarouter...end
start simulatio

In [65]:
random_mean, center_mean

(24.620133333333335, 25.057533333333332)

## Optimal Placement

In [31]:
opt_hubs = gpd.read_file('opt_hubs.gpkg')

In [32]:
#Popolate the geodataframe assigning, for each hub, its zone
zone_by_hub = []

for idx, row in opt_hubs.iterrows():
    street = row['geometry']
    # some hubs are outside the zones, this generate an error
    try:
        zone_row = zones_leuven['CODSEC'][np.where(zones_leuven['geometry'].contains(street))[0][0]]
        zone_by_hub.append(zone_row)
    except:
        zone_by_hub.append('NONE')

opt_hubs['ZONE'] = zone_by_hub

# drop all the 'NONE', street, all the street without a zone
opt_hubs.drop(list(np.where(opt_hubs['ZONE'] == 'NONE')[0]), axis = 0, inplace=True)

In [33]:
random_mean, center_mean = main_loop(opt_hubs)

-------------
iteration  1
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  2
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  3
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  4
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  5
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  6
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  7
start generation phase...end
start writing file...end
start duarouter...end
start simulation...end
-------------
iteration  8
start generation phase...end
start writing file...end
start duarouter...end
start simulatio

In [34]:
random_mean, center_mean 

(16.785889115646256, 17.12256598639456)